# Actividad árbol de decision 
En esta actividad, repetiremos el análisis para detección de cancer, usando esta vez Arboles de decisión

In [ ]:
import numpy as np
import pandas as pd
from plotnine import *

## Data

In [ ]:
from sklearn.datasets import load_breast_cancer
loadData = load_breast_cancer(as_frame=True)
data = loadData['data']
data.columns = data.columns.str.replace(' ','_')
target = loadData['target']
target = 1-target # para que "1" sea "Maligno"

Separamos los datos en un set de entrenamiento (`X_train` y `y_train`) para calibrar el modelo, y un set de test (`X_test`, `y_test`) para evaluarlo. (Ojo: no necesitamos normalizar los datos para este método)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=0)



## Parte 1:  Entrenar un modelo de Arbol de Decisión y evaluar su performance.

Genere un modelo de Arbol de Decisión y calibrelo para los datos de entrenamiento.  Con esto haga una predicción para los datos de `X_test`, y guarde estos resultados en la variable `prediccion`

In [ ]:
from sklearn.tree import DecisionTreeClassifier

AD = DecisionTreeClassifier()
AD = AD.fit(<Datos entrenamiento>, <Target entrenamiento>)
prediccion=NB.predict(<Datos test>)

Evalue el performance de su modelo, para las distintas métricas. Para esto, recuerde comparar los resultados reales `y_test` con los resultados predecidos `prediccion`.

Recuerde que puede usar distintas herramientas, por ejemplo:
- Accuracy : `mt.accuracy_score(<reales>,<prediccion>)` 
- Precision: `mt.precision_score(<reales>,<prediccion>)` 
- Recall: `mt.recall_score(<reales>,<prediccion>)` 
- F1: `mt.f1_score(<reales>,<prediccion>)` 
- Reporte general: `mt.classification_report(<reales>,<prediccion>)`
- Matriz confusión: `mt.confusion_matrix(<reales>,<prediccion>)`
- Matriz confusión (plot): `mt.ConfusionMatrixDisplay(mt.confusion_matrix(<reales>,<prediccion>)).plot()`

Recuerde F1= 2TP/(2TP+FN+FP), recall= TP/(TP+FP) y precision= TP/(TP+FN)

In [ ]:
from sklearn import metrics as mt
# Escriba aquí sus evaluaciones





## Parte 2 : Repetir la evaluación usando cross-validation
Para major certeza de lo obtenido, haremos un `KFold` (sugerencia: usar 10-fold), y usaremos `cross_validate` para evaluar cada una de las métricas, reportando su media y desviación estandar.  Para esto usamos los datos completos (o sea, no `X_train`, ...) ya que `cross_validate` se encargará de separar el conjunto train y test.

In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics as mt
from sklearn.model_selection import cross_validate


#Creamos un objeto de la clase KFold 
kf = KFold(n_skf = KFold(n_splits= <NUMERO DE FOLDS ELEGIDO> )

    
#Cree un objeto de la clase DecisionTreeClassifier. No necesita hacer .fit, cross_validate se encarga de eso
AD = DecisionTreeClassifier()


#Hacemos cross_validate para las distintas métricas. 
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(AD, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=metrica, cv=kf, return_train_score=True)    
    print ('Score',metrica,'train:', res['train_score'].mean(),'+-',res['train_score'].std())
    print ('Score',metrica,'test:', res['test_score'].mean(),'+-',res['test_score'].std())





## Parte 3 : Calibrar el modelo para distintas tamaños de arbol usando cross-validation

In [ ]:
#Creando el objeto con sus características 
from sklearn.model_selection import KFold
from sklearn import metrics as mt


# Defina la profundidad máxima a explorar
maxH= <SU PROFUNDIDAD MÁXIMA>

#Generando los k-fold
kf = KFold(n_splits=10)

# guardaremos los resultados acá
results = pd.DataFrame(columns=['depth','Tipo', 'Metrica', 'ScoreMean', 'ScoreStd'])

for k in range(1,maxH+1): #Ciclo for para buscar el mejor valor de max_depth
    #Cree un objeto de la clase KNeighborsClassifier con n_neighbors=k
    AD = DecisionTreeClassifier(max_depth=k)
    #Hacemos un ciclo for para las distintas métricas
    for metrica in ['accuracy', 'recall', 'precision', 'f1']:
        res = cross_validate(AD, <DATOS ORIGINALES>, <TARGET ORIGINAL>, scoring=metrica, cv=kf, return_train_score=True)    
        results.loc[len(results)] = [k,'train',metrica,res['train_score'].mean(),res['train_score'].std()]
        results.loc[len(results)] = [k,'test',metrica,res['test_score'].mean(),res['test_score'].std()]

results

Si todo resultó bien, grafique los resultados con el siguiente código:

In [ ]:
ggplot(results, aes(x='depth', color='Tipo'))\
+ geom_line(aes(y='ScoreMean', group='Tipo'))\
+ geom_errorbar(aes(ymin='ScoreMean-ScoreStd', ymax='ScoreMean+ScoreStd'))\
+ facet_wrap('Metrica')\
+ theme_bw()

**Pregunta**: ¿Cuál es la mejor profundidad máxima a su parecer?  Escoga una como "la mejor"

## Parte 4: Comparación con los métodos anteriores
Comparemos el performance obtenido para las distintas métricas, con los modelo KNN y GaussianNB anteriormente ajustados.  Repita el comando, usando `DecisionTreeClassifier` esta vez para la profundidad escogida

In [ ]:

#Creando el objeto con sus características 
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics as mt
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB


scaler = StandardScaler()
dataNorm = scaler.fit_transform(data)

results = pd.DataFrame(columns=['Metodo','Tipo', 'Metrica', 'ScoreMean', 'ScoreStd'])

#Cree un objeto de la clase KNeighborsClassifier con n_neighbors=k
KNNmodel = KNeighborsClassifier(n_neighbors=<SU NUMERO DE VECINOS PARA KNN>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(KNNmodel, dataNorm, target, cv=kf, return_train_score=True, scoring=metrica)    
    results.loc[len(results)] = ['KNN','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['KNN','test',metrica,res['test_score'].mean(),res['test_score'].std()]

NB = GaussianNB()
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(NB, data,target, cv=kf, return_train_score=True, scoring=metrica)
    results.loc[len(results)] = ['NB','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['NB','test',metrica,res['test_score'].mean(),res['test_score'].std()]

# Agregue la profundidad escogida
AD = DecisionTreeClassifier(max_depth=<PROFUNDIDAD_ESCOGIDA>)
for metrica in ['accuracy', 'recall', 'precision', 'f1']:
    res = cross_validate(AD, data,target, cv=kf, return_train_score=True, scoring=metrica)
    results.loc[len(results)] = ['AD','train',metrica,res['train_score'].mean(),res['train_score'].std()]
    results.loc[len(results)] = ['AD','test',metrica,res['test_score'].mean(),res['test_score'].std()]






In [ ]:
(ggplot(results)+
 aes(x='Metodo',y="ScoreMean",ymin="ScoreMean-ScoreStd",ymax="ScoreMean+ScoreStd",color='Tipo')+
 geom_point()+ geom_errorbar()+ 
 labs(y="Score")+
 facet_wrap('Metrica')+
 theme_bw()
)







## Parte 4: Interpretación de los resultados.
Visualicemos el árbol generado, así como el *importance feature* del modelo

In [ ]:
AD = DecisionTreeClassifier(max_depth=<PROFUNDIDAD_ESCOGIDA>)
AD.fit(data, target)

Graficamos el árbol.  Si es muy grande, puede cambiar el parámetro 'max_depth' de la función plot_tree (4ta línea) para ver solo los primeros niveles del árbol.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
plt.figure(figsize=(15,15)) #definiendo el tamaño de la figura
plot_tree(AD,filled=True,feature_names=data.columns, max_depth=100)
plt.show() #mostrando el árbol

Otra forma de interpretar el modelo es pidiendo los `.feature_importances_`.

In [ ]:
AD.feature_importances_

Gráfiquemoslos para ver el resultado

In [ ]:
importance = pd.DataFrame({'importance' : AD.feature_importances_}, index=data.columns)
importance_order = importance.sort_values('importance', ascending=True).index.to_list()

ggplot(importance, aes(x=importance.index, y='importance'))\
+ scale_x_discrete(limits=importance_order)\
+ geom_col() + coord_flip()


**Pregunta**: ¿Coincide con lo visto anteriormente para Naive-Bayes?